In [1]:
import robosuite as suite
from robosuite.wrappers.gym_wrapper import GymWrapper


[robosuite WARNING] No private macro file found! (macros.py:53)
[robosuite WARNING] It is recommended to use a private macro file (macros.py:54)
[robosuite WARNING] To setup, run: python C:\Users\ACERPC\PycharmProjects\robo_custom\robosuite\robosuite/scripts/setup_macros.py (macros.py:55)


In [2]:
# create environment instance
env = GymWrapper(suite.make(
    env_name="Lift", # try with other tasks like "Stack" and "Door"
    robots="UR5e",  # try with other robots like "Sawyer" and "Jaco"
    has_renderer=False,
    has_offscreen_renderer=True,
    use_object_obs=False,                   # don't provide object observations to agent
    use_camera_obs=True,
    camera_names="robot0_eye_in_hand",      # use "agentview" camera for observations
    camera_heights=84,                      # image height
    camera_widths=84,                       # image width
    reward_shaping=True,                    # use a dense reward signal for learning
    horizon = 200
), ['robot0_eye_in_hand_image'])

In [3]:
env.observation_space

Box(0, 255, (84, 84, 3), uint8)

In [ ]:
from stable_baselines3 import PPO
model = PPO("CnnPolicy", env, verbose=1)
model.learn(total_timesteps=1e2, progress_bar=True)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


Output()